In [1]:
import torch
import torch.nn.functional as F

from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

from synth import CrossEntropyDifferential

In [2]:
device = torch.device("cuda")

model_name = "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

cross_entropy_differential = CrossEntropyDifferential(model, tokenizer, device)

/home/tuccio/synth/menv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [65]:
Y = """Pizza is the best thing in the world, all the mozzarella and the tomato sauce on my margherita are amazing. 
My Italian friend Angelo told me that his favourite pizza is the quattroformaggi rossa with salsiccia. 
When I was a kid, I used to watch the pizza maker create his pizzas, he was from Romania but a very nice gentleman I have to say."""

prompt = f"""{Y}::TLDR::"""
input = tokenizer(prompt, return_tensors='pt', padding=True).to(device)

def generate_suggestion():
    return model.generate(
        **input,
        max_new_tokens=16,
        temperature=1.2,
        do_sample=True, # uncomment to enable fancy sampling of the output distribution
        no_repeat_ngram_size=2,
        num_return_sequences=200,
        pad_token_id=model.config.eos_token_id
    )

def rank_suggestion(suggestion):
    Ysuggestion = tokenizer.decode(suggestion, skip_special_tokens=True)
    suggestion = Ysuggestion.split("::TLDR::")[1]
    return Ysuggestion, cross_entropy_differential(suggestion, Y, diff=True)

suggestions = generate_suggestion()

n_gen = 10
gamma = -0.2
new_suggestions = []
barra = tqdm(total=n_gen)

while len(new_suggestions) < n_gen:
    suggestions = generate_suggestion()
    for suggestion in suggestions:
        Ysuggestion, rank = rank_suggestion(suggestion)
        if rank < gamma:
            barra.update(1)
            new_suggestions.append({Ysuggestion:rank.item()})

barra.close()

24it [00:05,  4.56it/s]                       


In [63]:
len(new_suggestions)

11

In [ ]:
n_gen = 1000
gamma = -0.2

new_suggestions = []

barra = tqdm(total=n_gen)

while len(new_suggestions) < n_gen:
    suggestion = generate_suggestion()
    rank = rank_suggestion(suggestion)
    if rank < gamma:
        barra.update(1)
        new_suggestions.append({f"{Y}::TLDR::{suggestion}":rank.item()})

barra.close()

In [3]:
Y = """Pizza is the best thing in the world, all the mozzarella and the tomato sauce on my margherita are amazing. 
My Italian friend Angelo told me that his favourite pizza is the quattroformaggi rossa with salsiccia. 
When I was a kid, I used to watch the pizza maker create his pizzas, he was from Romania but a very nice gentleman I have to say."""

prompt = f"""{Y}::TLDR::"""
input = tokenizer(prompt, return_tensors='pt', padding=True).to(device)

def generate_data():
    outputs = torch.tensor([])
    for i in range(1):
        torch.cuda.empty_cache()
        output = model.generate(
            input.input_ids,
            attention_mask=input.attention_mask,
            max_length=len(input.input_ids[0])+16,
            num_return_sequences=5,
            no_repeat_ngram_size=2,
            do_sample=True,
            temperature=1.2
        )
        outputs = torch.cat((outputs, output.to(torch.device('cpu'))), dim=0)

    results = []
    for i in output:
        answ = tokenizer.decode(i[len(input.input_ids[0]):], skip_special_tokens=True)
        results.append((
            cross_entropy_differential(answ, Y, diff=True).item(),
            answ
            ))
    best_result = min(results, key=lambda x: x[0])
    return best_result

In [8]:
model(input)

TypeError: list indices must be integers or slices, not tuple